# Data Processing for UI



In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler

In [19]:
from google.colab import drive
drive.mount('/content/drive/')

%cd drive/My Drive/mml_flood/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/mml_flood/'
/content/drive/My Drive/mml_flood


### Read data

In [20]:
#gdis data - geocoded disasters
gdis = pd.read_csv('data/disaster/pend-gdis-1960-2018-disasterlocations.csv')
# #emdat data - international disasters
emdat = pd.read_csv('data/disaster/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])

<ipython-input-20-53a980b4090f>:4: DtypeWarning: Columns (8,16,17,18,19,24,25,26,27,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  emdat = pd.read_csv('data/disaster/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])


  ### Filter emdat columns

In [21]:
print(emdat.columns)

Index(['Dis No', 'Year', 'Seq', 'Glide', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype',
       'Event Name', 'Country', 'ISO', 'Region', 'Continent', 'Location',
       'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response',
       'Appeal', 'Declaration', 'AID Contribution ('000 US$)', 'Dis Mag Value',
       'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time', 'River Basin',
       'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damages ('000 US$)', 'Insured Damages, Adjusted ('000 US$)',
       'Total Damages ('000 US$)', 'Total Damages, Adjusted ('000 US$)', 'CPI',
       'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Geo Locations'],
      dtype='object')


In [22]:
emdat['disasterno'] = emdat['Dis No'].str[:-4]  # Remove last 4 characters "-XYZ" area code

In [33]:
emdat_cols = ['disasterno', 'Year', 'Event Name',
              'Start Year', 'Start Month', 'Start Day',
              'End Year', 'End Month', 'End Day',
              "Total Damages, Adjusted ('000 US$)",
              'Total Deaths', 'Total Affected',
              "Reconstruction Costs, Adjusted ('000 US$)",
              'Disaster Subtype', 'OFDA Response', 'River Basin']

# Subset EMDAT to available columns (ignore missing ones if not present)
emdat = emdat[[col for col in emdat_cols if col in emdat.columns]]
print(emdat['OFDA Response'].count())  # check available data
print(emdat['River Basin'].count())    # check available data

1716
1312


In [34]:
print(emdat.columns)

Index(['disasterno', 'Year', 'Event Name', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Damages, Adjusted ('000 US$)', 'Total Deaths', 'Total Affected',
       'Reconstruction Costs, Adjusted ('000 US$)', 'Disaster Subtype',
       'OFDA Response', 'River Basin'],
      dtype='object')


In [35]:
print(gdis.columns)

Index(['id', 'country', 'iso3', 'gwno', 'year', 'geo_id', 'geolocation',
       'level', 'adm1', 'adm2', 'adm3', 'location', 'historical',
       'hist_country', 'disastertype', 'disasterno', 'latitude', 'longitude'],
      dtype='object')


# Merged cleaned emdat with gdis on disasterno

In [36]:
merged_df = pd.merge(emdat, gdis, on='disasterno', how='right')
merged_df.shape

(82885, 33)

In [37]:
# Drop duplicate records based on GDIS 'id'
merged_df = merged_df.drop_duplicates(subset=['id'])
print("Merged Data Shape:", merged_df.shape)

Merged Data Shape: (9924, 33)


In [38]:
# Create grid_id from GDIS coordinates
merged_df['lat'] = merged_df['latitude'].round().astype(int)
merged_df['lon'] = merged_df['longitude'].round().astype(int)
merged_df['grid_id'] = merged_df['lat'].astype(str) + "_" + merged_df['lon'].astype(str)


In [39]:
merged_df.head()

,disasterno,Year,Event Name,Start Year,Start Month,Start Day,End Year,End Month,End Day,"Total Damages, Adjusted ('000 US$)",...,adm3,location,historical,hist_country,disastertype,latitude,longitude,lat,lon,grid_id
0,2009-0631,2009.0,NaN,2009.0,12.0,27.0,2010.0,1.0,8.0,NaN,...,Ana E Malit,Ana E Malit,0,NaN,flood,42.020948,19.418317,42,19,42_19
2,2001-0146,2001.0,NaN,2001.0,4.0,2.0,2001.0,4.0,9.0,NaN,...,Onjiva,Onjiva,0,NaN,flood,-17.093484,15.665758,-17,16,-17_16
3,2009-0092,2009.0,NaN,2009.0,3.0,1.0,2009.0,4.0,16.0,NaN,...,Evale,Evale,0,NaN,flood,-16.531533,15.773987,-17,16,-17_16
18,2010-0105,2010.0,NaN,2010.0,3.0,1.0,2010.0,3.0,17.0,NaN,...,Evale,Evale,0,NaN,flood,-16.531533,15.773987,-17,16,-17_16
20,1995-0082,1995.0,NaN,1995.0,5.0,15.0,1995.0,5.0,15.0,1422569.0,...,Hatiya,Hatiya,0,NaN,storm,22.291591,91.065456,22,91,22_91


In [30]:
# df_all = merged_df.copy()

# agg_all = df_all.groupby('grid_id').agg(
#     all_count=('disasterno','count'),
#     all_damages=("Total Damages, Adjusted ('000 US$)", 'sum'),
#     lat=('lat','first'),
#     lon=('lon','first'),
#     primary_admin1=('adm1', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
#     unique_locations=('location', lambda x: ', '.join(sorted(x.unique()))),
#     all_deaths=('Total Deaths','sum'),
#     predominant_disaster=('disastertype', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
#     all_disastertypes=('disastertype', lambda x: ', '.join(sorted(x.unique())))
# ).reset_index()

# # Create a helpful string summary
# agg_all['event_summary'] = agg_all.apply(
#     lambda row: f"""In total {row['all_count']} events across all disasters;
#     predominant disaster: {row['predominant_disaster']};
#     types of disasters: {row['all_disastertypes']};
#     total damages: {row['all_damages']} ('000 US$);
#     total deaths: {row['all_deaths']}.""",
#     axis=1
# )

# print("All-disaster summary shape:", agg_all.shape)
# agg_all.head()

All-disaster summary shape: (2827, 11)


,grid_id,all_count,all_damages,lat,lon,primary_admin1,unique_locations,all_deaths,predominant_disaster,all_disastertypes,event_summary
0,-10_-36,1,531.0,-10,-36,Alagoas,Maceio,28.0,flood,flood,In total 1 events across all disasters;\n p...
1,-10_-37,6,895928.0,-10,-37,Alagoas,Alagoas,511.0,flood,"flood, landslide",In total 6 events across all disasters;\n p...
2,-10_-48,1,0.0,-10,-48,Tocantins,Tocantins,0.0,drought,drought,In total 1 events across all disasters;\n p...
3,-10_-66,1,0.0,-10,-66,Pando,Nueva Esperanza,25.0,flood,flood,In total 1 events across all disasters;\n p...
4,-10_-75,2,0.0,-10,-75,Pasco,Pasco,418.0,extreme temperature,extreme temperature,In total 2 events across all disasters;\n p...
...,...,...,...,...,...,...,...,...,...,...,...
2822,9_78,1,0.0,9,78,Tamil Nadu,Virudhunagar district,8.0,flood,flood,In total 1 events across all disasters;\n p...
2823,9_79,2,0.0,9,79,Tamil Nadu,"Ramanathapurum, Rameswaram",507.0,storm,storm,In total 2 events across all disasters;\n p...
2824,9_80,3,1354808.0,9,80,Kilinochchi,"Vavuniya, Kilinochchi district , Northern",212.0,flood,flood,In total 3 events across all disasters;\n p...
2825,9_81,4,325863.0,9,81,Trincomalee,"Mullaitivu district, Trincomalee",249.0,flood,"flood, storm",In total 4 events across all disasters;\n p...


# Add duration feature (in days)

In [40]:
def compute_duration(row):
    try:
        start_date = pd.Timestamp(
            year=int(row['Start Year']),
            month=int(row['Start Month']),
            day=int(row['Start Day'])
        )
        end_date = pd.Timestamp(
            year=int(row['End Year']),
            month=int(row['End Month']),
            day=int(row['End Day'])
        )
        return (end_date - start_date).days # duration computed in DAYS
    except:
        # If something is missing or weird, we return NaN
        return np.nan

# We'll apply this function to our merged_df
merged_df['duration'] = merged_df.apply(compute_duration, axis=1)
merged_df.head()

,disasterno,Year,Event Name,Start Year,Start Month,Start Day,End Year,End Month,End Day,"Total Damages, Adjusted ('000 US$)",...,location,historical,hist_country,disastertype,latitude,longitude,lat,lon,grid_id,duration
0,2009-0631,2009.0,NaN,2009.0,12.0,27.0,2010.0,1.0,8.0,NaN,...,Ana E Malit,0,NaN,flood,42.020948,19.418317,42,19,42_19,12.0
2,2001-0146,2001.0,NaN,2001.0,4.0,2.0,2001.0,4.0,9.0,NaN,...,Onjiva,0,NaN,flood,-17.093484,15.665758,-17,16,-17_16,7.0
3,2009-0092,2009.0,NaN,2009.0,3.0,1.0,2009.0,4.0,16.0,NaN,...,Evale,0,NaN,flood,-16.531533,15.773987,-17,16,-17_16,46.0
18,2010-0105,2010.0,NaN,2010.0,3.0,1.0,2010.0,3.0,17.0,NaN,...,Evale,0,NaN,flood,-16.531533,15.773987,-17,16,-17_16,16.0
20,1995-0082,1995.0,NaN,1995.0,5.0,15.0,1995.0,5.0,15.0,1422569.0,...,Hatiya,0,NaN,storm,22.291591,91.065456,22,91,22_91,0.0


In [41]:
##################################### THIS WAS initially MIN-MAX normalization #########################################

# We want a fancy way to measure how "bad" a disaster is, based on:
# 1) Frequency
# 2) Deaths
# 3) Cost (damages)

# Min-Max normalization is too sensitive to skewed values. Min-Max with z-score normalization is also inappropriate since our data is not normal

# Better to use log normalization to handle large values AND robust scaling


def compute_severity_score(
    df,
    count_col='count',
    deaths_col='total_deaths',
    damages_col='total_damages',
    weight_count=0.2,
    weight_deaths=0.5,
    weight_damages=0.3
):

    # 1) Replace NaNs
    # 2) Log(1 + x) to shrink big vals
    # 3) Use RobustScaler to handle outliers
    # 4) Weighted sum (like a recipe with different ingredients)
    # 5) Rescale from 0 to 100 (least to most severe)

    df_out = df.copy() # copy into new df

    # 1) Fill missing with 0
    df_out[count_col]   = df_out[count_col].fillna(0)
    df_out[deaths_col]  = df_out[deaths_col].fillna(0)
    df_out[damages_col] = df_out[damages_col].fillna(0)

    # 2) log transform
    df_out['count_log']   = np.log1p(df_out[count_col])
    df_out['deaths_log']  = np.log1p(df_out[deaths_col])
    df_out['damages_log'] = np.log1p(df_out[damages_col])

    # 3) robust scaling
    scaler = RobustScaler()
    scaled = scaler.fit_transform(df_out[['count_log','deaths_log','damages_log']])
    df_out['count_scaled']   = scaled[:,0]
    df_out['deaths_scaled']  = scaled[:,1]
    df_out['damages_scaled'] = scaled[:,2]

    # 4) Weighted sum
    df_out['severity_score'] = (
        df_out['count_scaled']   * weight_count +
        df_out['deaths_scaled']  * weight_deaths +
        df_out['damages_scaled'] * weight_damages
    )

    # 5) Rescale 0-100
    min_s = df_out['severity_score'].min()
    max_s = df_out['severity_score'].max()
    df_out['severity_score_normalized'] = (
        (df_out['severity_score'] - min_s) / (max_s - min_s + 1e-9) * 100
    )

    # drop temporary columns
    df_out.drop(
        columns=['count_log','deaths_log','damages_log',
                 'count_scaled','deaths_scaled','damages_scaled'],
        inplace=True
    )

    return df_out


# Replicate aggregator for all disasters

In [80]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

def aggregator(merged_df, disaster_keyword, dataset_end_year=2018):
    """
    1) Filters by disaster keyword
    2) Groups by 'grid_id' to compute:
       - count: how many events
       - total_damages
       - total_deaths
       - total_affected
       - avg_duration
       - freq_5yr, freq_10yr, freq_20yr: how many events in last 5/10/20 years -> based on end year
       - top_month: the single month with the highest event count
       - monthly_dist: a string listing how many events in each month over the years
       - risk_slope on year/counts
       - severity_score: log+robust scaling for count, deaths, damages
    3) Renames all columns to disaster_keyword_... (e.g. "flood_count", "flood_risk_slope")
    """

    # 1. Subset of disaster type
    df_dis = merged_df[
        merged_df['disastertype'].str.lower().str.contains(disaster_keyword.lower(), na=False, regex=False)
    ].copy() # Copy into new df just the specified disaster

    # 2.

    # (i) Frequency counters in last 5/10/20 years
    def frequency_counters(grp):

        five   = grp[grp['year'] >= (dataset_end_year - 5)]
        ten    = grp[grp['year'] >= (dataset_end_year - 10)]
        twenty = grp[grp['year'] >= (dataset_end_year - 20)]
        return pd.Series({
            'freq_5yr':  len(five),
            'freq_10yr': len(ten),
            'freq_20yr': len(twenty)
        })

    # (ii) Monthly profile
    def monthly_profile(grp):

        month_counts = grp['Start Month'].value_counts().sort_index() # Count events with specified start month
        if len(month_counts) == 0:
            return pd.Series({'top_month': np.nan, 'monthly_dist': ''})
        top_m = month_counts.idxmax()                         # month with most events started
        dist_str = ";".join(f"{m}:{cnt}" for m,cnt in month_counts.items())
        return pd.Series({'top_month': top_m, 'monthly_dist': dist_str})  # Distribution of disaster for each month

    # (iii) Slope of events over time
    def slope_of_events(grp):
        """
        Fit a simple linear regression on YEAR/COUNTS
        POSITIVE -> increasing over time
        NEGATIVE -> decreasing over time
        """
        per_year = grp.groupby('year').size().reset_index(name='count')
        if len(per_year) < 2:
            return 0.0  # not enough data to compute slope
        X = per_year[['year']]
        y = per_year['count']
        model = LinearRegression().fit(X, y)
        return model.coef_[0]

    # 3. Aggregations for each grid_id

    # Basic aggregator: count, damages, deaths, etc.
    base_agg = df_dis.groupby('grid_id').agg(
        count=('disasterno','count'),
        total_damages=("Total Damages, Adjusted ('000 US$)", 'sum'),
        total_deaths=('Total Deaths','sum'),
        total_affected=('Total Affected','sum'),
        avg_duration=('duration','mean')
    ).reset_index()

    # 5/10/20 year counts (use group_keys=False to avoid the deprecation warning)
    freq_df = df_dis.groupby('grid_id', group_keys=False).apply(frequency_counters).reset_index()

    # Monthly distribution
    month_df = df_dis.groupby('grid_id', group_keys=False).apply(monthly_profile).reset_index()

    # Risk slope
    slope_df = df_dis.groupby('grid_id', group_keys=False).apply(slope_of_events).reset_index(name='risk_slope')

    # Merge results
    merged_features = (base_agg
                       .merge(freq_df, on='grid_id', how='left')
                       .merge(month_df, on='grid_id', how='left')
                       .merge(slope_df, on='grid_id', how='left'))

    # 4. Compute severity score using the normalization function
    merged_features = compute_severity_score(
        merged_features,
        count_col='count',
        deaths_col='total_deaths',
        damages_col='total_damages',
        weight_count=0.2,
        weight_deaths=0.5,
        weight_damages=0.3
    )

    # 5. Rename columns to match disaster keyword
    rename_map = {}
    for col in merged_features.columns:
        if col != 'grid_id':
            rename_map[col] = f"{disaster_keyword}_{col}"
    merged_features.rename(columns=rename_map, inplace=True)

    return merged_features


# Maintenance work to clean dataframes, round up numbers etc.

In [81]:
def clean_df(df, prefix):

    # 1) Reindex columns in preferred order
    order = [
        "grid_id",
        f"{prefix}_count",
        f"{prefix}_freq_5yr",
        f"{prefix}_freq_10yr",
        f"{prefix}_freq_20yr",
        f"{prefix}_total_damages",
        f"{prefix}_total_deaths",
        f"{prefix}_total_affected",
        f"{prefix}_avg_duration",
        f"{prefix}_top_month",
        f"{prefix}_monthly_dist",
        f"{prefix}_risk_slope",
        f"{prefix}_severity_score",
        f"{prefix}_severity_score_normalized",
    ]
    df = df.reindex(columns=[col for col in order if col in df.columns])

    # 2) Round numeric columns
    numeric_cols = [
        f"{prefix}_count",
        f"{prefix}_freq_5yr",
        f"{prefix}_freq_10yr",
        f"{prefix}_freq_20yr",
        f"{prefix}_total_damages",
        f"{prefix}_total_deaths",
        f"{prefix}_total_affected",
        f"{prefix}_avg_duration",
        f"{prefix}_risk_slope",
        f"{prefix}_severity_score",
        f"{prefix}_severity_score_normalized",
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].round(2) # round to 2 decimals

    # 3) Fill missing in string columns with 'NA'
    str_cols = [f"{prefix}_top_month", f"{prefix}_monthly_dist"]
    for col in str_cols:
        if col in df.columns:
            df[col] = df[col].fillna("NA")

    # 4) Convert numeric months in 'top_month' into strings
    top_month_col = f"{prefix}_top_month"
    if top_month_col in df.columns:
        df[top_month_col] = pd.to_numeric(df[top_month_col], errors="coerce").round(0)
        month_map = {
            1: "January",  2: "February",  3: "March",     4: "April",
            5: "May",      6: "June",      7: "July",      8: "August",
            9: "September",10: "October",  11: "November", 12: "December"
        }
        df[top_month_col] = df[top_month_col].map(month_map).fillna("Unknown")

    # 5) Parse "3.0:2" -> "March:2"
    monthly_dist_col = f"{prefix}_monthly_dist"
    if monthly_dist_col in df.columns:
        month_map = {
            1: "January",  2: "February",  3: "March",     4: "April",
            5: "May",      6: "June",      7: "July",      8: "August",
            9: "September",10: "October",  11: "November", 12: "December"
        }
        def parse_monthly_dist(md):
            # If "NA", just return it
            if md == "NA" or md.strip() == "":
                return md
            # e.g. "3.0:2;4.0:1" -> split by ";"
            chunks = md.split(";")
            output_chunks = []
            for chunk in chunks:
                # chunk might be "3.0:2"
                if ":" in chunk:
                    month_str, count_str = chunk.split(":")
                    # parse the month float -> int
                    try:
                        m_float = float(month_str)
                        m_int = int(round(m_float))
                        m_name = month_map.get(m_int, "Unknown")
                        output_chunks.append(f"{m_name}:{count_str}")
                    except:
                        # fallback if parse fails
                        output_chunks.append(chunk)
                else:
                    output_chunks.append(chunk)
            return ";".join(output_chunks)

        df[monthly_dist_col] = df[monthly_dist_col].apply(parse_monthly_dist)

    return df


# Run through desired disasters and apply the master aggregator

In [82]:
# Extract unique disasters from disastertype columns
unique_disasters = (merged_df["disastertype"].dropna().unique().tolist()                 )

print(unique_disasters)


['flood', 'storm', 'earthquake', 'extreme temperature ', 'landslide', 'volcanic activity', 'drought', 'mass movement (dry)']


In [83]:
unique_disasters = merged_df["disastertype"].dropna().unique().tolist()
df_dict = {}
for d in unique_disasters:
    df_temp = aggregator(merged_df, d)
    df_temp = clean_df(df_temp, d.lower().replace(" ", "_"))
    df_dict[d] = df_temp


<ipython-input-80-4adbb56ea08b>:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  freq_df = df_dis.groupby('grid_id', group_keys=False).apply(frequency_counters).reset_index()
<ipython-input-80-4adbb56ea08b>:81: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  month_df = df_dis.groupby('grid_id', group_keys=False).apply(monthly_profile).reset_index()
<ipython-input-80-4adbb56ea08b>:84: DeprecationWarning: Data

In [84]:
for disaster_name, df_agg in df_dict.items():
    print(f"***** {disaster_name} *****")
    display(df_agg.head(10))

# NOTE: risk slope needs to be revised

***** flood *****


,grid_id,flood_count,flood_freq_5yr,flood_freq_10yr,flood_freq_20yr,flood_total_damages,flood_total_deaths,flood_total_affected,flood_avg_duration,flood_top_month,flood_monthly_dist,flood_risk_slope,flood_severity_score,flood_severity_score_normalized
0,-10_-36,1,0,0,1,531.0,28.0,2254.0,1.0,June,June:1,0.0,0.27,25.08
1,-10_-37,5,0,1,1,777910.0,451.0,2110149.0,4.4,March,March:2;April:2;July:1,0.0,1.18,59.82
2,-10_-66,1,0,0,0,0.0,25.0,20000.0,NaN,February,February:1,0.0,0.09,18.26
3,-10_124,1,0,1,1,0.0,16.0,200.0,3.0,November,November:1,0.0,0.03,15.88
4,-10_14,2,0,0,2,0.0,1.0,12000.0,30.5,January,January:1;December:1,0.0,-0.17,8.37
5,-10_15,1,0,0,1,15736.0,31.0,70000.0,7.0,January,January:1,0.0,0.37,28.98
6,-10_160,3,2,3,3,29797.0,49.0,78244.0,10.0,January,January:1;April:1;August:1,0.0,0.65,39.78
7,-10_31,1,0,0,1,34410.0,0.0,1300000.0,0.0,February,February:1,0.0,-0.12,10.34
8,-10_34,5,2,3,5,148.0,40.0,69404.0,16.8,April,January:2;April:3,0.0,0.60,37.90
9,-11_-42,1,0,0,0,0.0,50.0,270000.0,NaN,February,February:1,0.0,0.19,22.03


***** storm *****


,grid_id,storm_count,storm_freq_5yr,storm_freq_10yr,storm_freq_20yr,storm_total_damages,storm_total_deaths,storm_total_affected,storm_avg_duration,storm_top_month,storm_monthly_dist,storm_risk_slope,storm_severity_score,storm_severity_score_normalized
0,-10_151,2,0,1,1,3014402.0,16.0,47340.0,1.50,February,February:1;May:1,0.0,0.26,31.16
1,-10_160,5,1,2,2,49443.0,109.0,239500.0,0.00,January,January:2;March:1;May:1;November:1,0.0,0.68,45.29
2,-10_57,1,0,0,0,0.0,1.0,351.0,0.00,December,December:1,0.0,-0.55,3.89
3,-11_-37,1,0,0,0,0.0,10.0,100.0,0.00,June,June:1,0.0,-0.26,13.47
4,-12_160,1,0,0,1,0.0,0.0,275.0,2.00,January,January:1,0.0,-0.66,0.00
5,-12_169,4,0,0,2,0.0,0.0,1910.0,0.33,March,March:1;June:1;December:1,0.0,-0.40,8.92
6,-12_34,1,0,0,1,0.0,11.0,8.0,0.00,December,December:1,0.0,-0.25,13.96
7,-12_39,1,0,1,1,0.0,77.0,85015.0,0.00,February,February:1,0.0,0.06,24.47
8,-12_43,1,0,0,0,14026.0,2.0,35000.0,0.00,February,February:1,0.0,-0.26,13.40
9,-12_44,5,1,1,2,95481.0,57.0,89863.0,0.80,January,January:3;March:1;April:1,0.0,0.59,42.19


***** earthquake *****


...

,grid_id
0,-11_166
1,-12_43
2,-14_167
3,-15_168
4,-16_-71
5,-16_168
6,-17_-71
7,-1_-78
8,-1_-79
9,-1_100


***** drought *****


,grid_id,drought_count,drought_freq_5yr,drought_freq_10yr,drought_freq_20yr,drought_total_damages,drought_total_deaths,drought_total_affected,drought_avg_duration,drought_top_month,drought_monthly_dist,drought_risk_slope,drought_severity_score,drought_severity_score_normalized
0,-10_-48,1,0,0,1,0.0,0.0,1000000.0,NaN,October,October:1,0.0,0.00,0.00
1,-10_124,1,0,0,1,1473.0,0.0,15000.0,NaN,August,August:1,0.0,0.20,3.22
2,-10_151,1,0,0,0,0.0,60.0,500000.0,NaN,September,September:1,0.0,2.06,33.65
3,-11_15,2,0,0,2,0.0,58.0,25000.0,NaN,January,January:1;September:1,0.0,2.12,34.71
4,-12_-42,1,1,1,1,5723031.0,0.0,27000000.0,NaN,January,January:1,0.0,0.42,6.86
5,-12_160,1,1,1,1,0.0,0.0,67000.0,NaN,September,September:1,0.0,0.00,0.00
6,-12_17,1,0,0,0,0.0,0.0,80000.0,NaN,July,July:1,0.0,0.00,0.00
7,-12_34,1,0,0,1,0.0,0.0,520000.0,NaN,October,October:1,0.0,0.00,0.00
8,-12_43,1,0,0,0,0.0,0.0,0.0,NaN,August,August:1,0.0,0.00,0.00
9,-14_34,1,0,0,0,0.0,0.0,7000000.0,NaN,April,April:1,0.0,0.00,0.00


***** mass movement (dry) *****


,grid_id
0,-12_-77
1,-3_-78
2,-6_146
3,-7_147
4,-9_-78
5,0_100
6,10_123
7,15_-91
8,15_-92
9,16_121


### TODO: fix RISK SLOPE, avg duration for drought; drop duration for earthquake events